In [1]:
import exmp
import os.path
import qiime2
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

from qiime2.plugins.diversity.actions import filter_distance_matrix
from qiime2.plugins.longitudinal.actions import first_distances
import scipy.stats

from statsmodels.stats.multitest import multipletests

In [2]:

def microbiome_performance_correlations(project, 
                                        time_column, 
                                        baseline_time_value,
                                        performance_metrics,
                                        week,
                                        sample_metadata,
                                        data_dir,
                                        output_dir):

    results = []
    uu = ("unweighted UniFrac", os.path.join(data_dir, "unweighted_unifrac_distance_matrix.qza"))
    wu = ("weighted UniFrac", os.path.join(data_dir, "weighted_unifrac_distance_matrix.qza"))
    bc = ("Bray-Curtis", os.path.join(data_dir, "bray_curtis_distance_matrix.qza"))
    bj = ("Jaccard", os.path.join(data_dir, "jaccard_distance_matrix.qza"))
    
    where = "[project]='%s' and [exclude]='no'" % project
    ids_to_keep = sample_metadata.get_ids(where=where)
    sample_metadata = sample_metadata.filter_ids(ids_to_keep=ids_to_keep)


    metadata_to_merge = []
    distance_columns = []

    for metric, dm_fp in [uu, wu, bc, bj]:
        dm = qiime2.Artifact.load(dm_fp)
        dm = filter_distance_matrix(dm, metadata=sample_metadata).filtered_distance_matrix
        # add distances to baseline to sample metadata
        dists_to_baselines = first_distances(distance_matrix=dm, metadata=sample_metadata, state_column=time_column, 
                                             individual_id_column='subject-id', baseline=baseline_time_value, 
                                             replicate_handling='random').first_distances
        dists_to_baselines = dists_to_baselines.view(qiime2.Metadata).get_column('Distance').to_dataframe()
        column_name = '%s distance (%s %d to %s)' % (metric, time_column, baseline_time_value, week)
        dists_to_baselines = dists_to_baselines.rename(columns = {'Distance' : column_name})

        metadata_to_merge.append(qiime2.Metadata(dists_to_baselines))
        distance_columns.append(column_name)

    for e in metadata_to_merge:
        sample_metadata = sample_metadata.merge(e)
    data = sample_metadata.to_dataframe()

    for distance_column in distance_columns:
        for performance_metric in performance_metrics:
            where = "[%s]='%s'" % (time_column, week)
            ids_to_keep = sample_metadata.get_ids(where=where)
            sample_metadata_subsample = sample_metadata.filter_ids(ids_to_keep=ids_to_keep).to_dataframe()
            sample_metadata_subsample = sample_metadata_subsample[[distance_column, performance_metric]].dropna().astype(np.float)
            rho, p = scipy.stats.spearmanr(sample_metadata_subsample[[distance_column, performance_metric]])
            results.append((project, distance_column, performance_metric, rho, p, sample_metadata_subsample.shape[0]))
            fig_fp = '%s/%s-%s-%s.pdf' % (output_dir, project, distance_column, performance_metric)
            sns.scatterplot(sample_metadata_subsample[distance_column], 
                            sample_metadata_subsample[performance_metric]).get_figure().savefig(fig_fp)
            plt.clf()
    df = pd.DataFrame(results, columns=['project', 'distance', 'performance metric', 'Spearman rho', 'Spearman p-value', 'sample size'])
    df['Spearman q-value'] = multipletests(df['Spearman p-value'])[1]
    df.to_csv('%s/%s-%s.csv' % (output_dir, project, week))
    return df

In [3]:
output_dir = '../data/exmp1-and-exmp2/cm/microbiome-performance-correlations/'

df_exmp1 = microbiome_performance_correlations(
    'exmp1', 'week', 1.0, ['RER-change', 'VO2max-change'], 
    '5.0', exmp.load_sample_metadata(), exmp.cm_path, output_dir)
df_exmp2 = microbiome_performance_correlations(
    'exmp2', 'week', 1.0, ['bench-press-change', 'row-change', '3RM-squat-change'], 
    '5.0', exmp.load_sample_metadata(), exmp.cm_path, output_dir)
df_exmp1 = microbiome_performance_correlations(
    'exmp1', 'week', 1.0, ['RER-change', 'VO2max-change'], 
    '6.0', exmp.load_sample_metadata(), exmp.cm_path, output_dir)
df_exmp2 = microbiome_performance_correlations(
    'exmp2', 'week', 1.0, ['bench-press-change', 'row-change', '3RM-squat-change'], 
    '6.0', exmp.load_sample_metadata(), exmp.cm_path, output_dir)